In [47]:
#define a new DFT which is unitary
#NOTE: in Beals' ['97] he normalizes by \sqrt{d_\lambda/n!}
#but also notes that a basis change is an equivalence relation on rep'ns
#and each equivalence class contains a unitary representatione
#if each rep'n \rho \in \hat{G} is unitary, then the transformation is unitary
#these representations are not unitary
#to make them unitary, use Weyl's unitary trick
#OPTION 1: use the formula P = \int_G \rho(g)\rho(g)^* dg, and take a square root to find Q s.t. P = Q^2
#OPTION 2: define a new invariant inner product compute an orthonormal basis w.r.t. 
#this inner product by computing the Gram matrix A and using A.gram_schmidt()

In [48]:
#compute a field which contains all the square roots required
def containing_field(SGA):
    required_square_roots = []
    for partition in Partitions(SGA.group().degree()):
        specht_module = SGA.specht_module(partition)
        rho = specht_module.representation_matrix
        group_size = SGA.group().cardinality()
        P = (1/group_size)*sum(rho(g)*rho(g).conjugate().transpose() for g in SGA.group())
        d, L = P.eigenmatrix_left()
        required_square_roots += [specht_module.dimension(),SGA.group().cardinality()] + d.diagonal()
    required_square_roots = flatten([[QQ(q).numerator(),QQ(q).denominator()] if q in QQ else q for q in required_square_roots])
    K = SGA.base_ring()
    for n in set(required_square_roots):
        R = PolynomialRing(K, 'x')
        x = R.gen()
        if (x**2 - n).is_irreducible():
            gen_name = "sqrt"+str(n).replace("/","over")
            K = K.extension(sqrt(n).minpoly(),names=gen_name)
    return K

In [49]:
#find the change-of-basis matrix Q making \rho(g) unitary for all g \in G
from sage.matrix.special import diagonal_matrix
from sage.misc.functional import sqrt
def unitary_change_of_basis(SGA,partition,K):
    rho = SGA.specht_module(partition).representation_matrix
    group_size = SGA.group().cardinality()
    P = (1/group_size)*sum(rho(g)*rho(g).conjugate().transpose() for g in SGA.group())
    d, L = P.eigenmatrix_left()
    return L.inverse() * diagonal_matrix([sqrt(K(a)) for a in d.diagonal()]) * L

In [50]:
#define the Fourier coefficient at the representation specht_module
#which is the Specht module corresponding to partition
def hat(f,partition,SGA,K,unitary=False):
    specht_module = SGA.specht_module(partition)
    rho = specht_module.representation_matrix
    if unitary:
        Q = unitary_change_of_basis(SGA,partition,K)
        unitary_factor = specht_module.dimension()/SGA.group().cardinality()
        sqrt_unitary_factor = sqrt(K(unitary_factor))
        return sqrt_unitary_factor*sum(f(g)*Q.inverse()*rho(g)*Q for g in SGA.group())
    else:
        return sum(f(g)*rho(g) for g in SGA.group())

In [51]:
#define the delta function delta_s(t) = {1 if s == t, 0 otherwise}
delta = lambda s: lambda t: 1 if t == s else 0

In [52]:
#for each basis element g \in G compute the Fourier coefficients \hat{\delta_g}(partition) for all partitions
from sage.misc.flatten import flatten
def unitary_dft(SGA):
    K = containing_field(SGA)
    return matrix(K,[flatten([hat(delta(g),partition,SGA,K,unitary=True).list() for partition in Partitions(SGA.group().degree())]) for g in SGA.group()]).transpose()

In [53]:
n = 3

In [54]:
SGA = SymmetricGroupAlgebra(QQ,n)

In [55]:
K = containing_field(SGA); K

Number Field in sqrt3 with defining polynomial x^2 - 3 over its base field

In [56]:
G = SGA.group()

In [57]:
SGA_dft = SGA.dft(); SGA_dft

[   1    1    1    1    1    1]
[   1  1/2   -1 -1/2 -1/2  1/2]
[   0  3/4    0  3/4 -3/4 -3/4]
[   0    1    0   -1    1   -1]
[   1 -1/2    1 -1/2 -1/2 -1/2]
[   1   -1   -1    1    1   -1]

In [58]:
#check if A*A^T == Id. it's not, but the columns are orthonormal
SGA_dft*SGA_dft.transpose()

[  6   0   0   0   0   0]
[  0   3   0   0   0   0]
[  0   0 9/4   0   0   0]
[  0   0   0   4   0   0]
[  0   0   0   0   3   0]
[  0   0   0   0   0   6]

In [59]:
partition = Partitions(SGA.group().degree())[1]; partition

[2, 1]

In [60]:
specht_module = SGA.specht_module(partition)

In [61]:
rho = specht_module.representation_matrix

In [62]:
group_size = SGA.group().cardinality()

In [63]:
P = (1/group_size)*sum(rho(g)*rho(g).conjugate().transpose() for g in SGA.group()); P.base_ring()

Rational Field

In [64]:
Q = unitary_change_of_basis(SGA,partition,K); Q

[-1/6*sqrt2*sqrt3 - 1/2*sqrt2  1/6*sqrt2*sqrt3 - 1/2*sqrt2]
[ 1/6*sqrt2*sqrt3 - 1/2*sqrt2 -1/6*sqrt2*sqrt3 - 1/2*sqrt2]

In [65]:
unitary_factor = specht_module.dimension()/SGA.group().cardinality(); unitary_factor

1/3

In [66]:
K

Number Field in sqrt3 with defining polynomial x^2 - 3 over its base field

In [67]:
G[3]

[2, 3, 1]

In [68]:
U_rho = Q.inverse()*rho(G[3])*Q; U_rho

[      -1/2  1/2*sqrt3]
[-1/2*sqrt3       -1/2]

In [69]:
U_rho*U_rho.conjugate().transpose()

[1 0]
[0 1]

In [70]:
unitary_dft(SGA)

[-1/6*sqrt2*sqrt3 -1/6*sqrt2*sqrt3 -1/6*sqrt2*sqrt3 -1/6*sqrt2*sqrt3 -1/6*sqrt2*sqrt3 -1/6*sqrt2*sqrt3]
[      -1/3*sqrt3                0             -1/2        1/6*sqrt3        1/6*sqrt3              1/2]
[               0       -1/3*sqrt3        1/6*sqrt3             -1/2              1/2        1/6*sqrt3]
[               0       -1/3*sqrt3        1/6*sqrt3              1/2             -1/2        1/6*sqrt3]
[      -1/3*sqrt3                0              1/2        1/6*sqrt3        1/6*sqrt3             -1/2]
[-1/6*sqrt2*sqrt3  1/6*sqrt2*sqrt3  1/6*sqrt2*sqrt3 -1/6*sqrt2*sqrt3 -1/6*sqrt2*sqrt3  1/6*sqrt2*sqrt3]

In [71]:
U_dft = unitary_dft(SGA); U_dft*U_dft.conjugate().transpose()

[1 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 1 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 1]

In [72]:
#check that the DFT is unitary
(U_dft*U_dft.transpose()) == identity_matrix(SGA.group().cardinality())

True

In [73]:
#QUESTION: what are the eigenvalues?
#need to be careful as characteristic polynomial is degree n! over K, so [L:QQ] = [L:K]*[K:QQ] = n!*[K:QQ]
#for n=3, [L:QQ] = n!*[K:QQ] = 6*4 = 24

In [74]:
U_dft.charpoly()

x^6 + (-1/3*sqrt3 - 1/2)*x^5 + ((-1/6*sqrt2 + 1/6)*sqrt3 - 1/3*sqrt2 - 1/3)*x^4 + ((1/3*sqrt2 + 1/3)*sqrt3 + 1/2*sqrt2)*x^3 + ((-1/6*sqrt2 + 1/6)*sqrt3 - 1/3*sqrt2 - 1/3)*x^2 + (-1/3*sqrt3 - 1/2)*x + 1

In [75]:
#ISSUE: the splitting field appears to be high degree, and the coefficients are large
#the defining polynomial is monic with integer coefficients
#since the eigenvalues lie on the unit circle, a result of Kroenecker implies they are roots of unity
if len(U_dft.charpoly().factor()) != U_dft.charpoly().degree():
    L.<a> = U_dft.charpoly().splitting_field(); L
else:
    L = K

In [97]:
L.absolute_degree()

192

In [99]:
roots_defining_poly = L.polynomial().roots(ring=CC);

In [104]:
[abs(root[0]) for root in roots_defining_poly if abs(root[0]) <= 1]

[0.955110617391679, 0.955110617391679]

In [107]:
eigs[0].minpoly()

x^24 + 2*x^23 - 1/2*x^22 - 17/6*x^21 - 215/144*x^20 - 1/6*x^19 + 509/216*x^18 + 11/3*x^17 - 35/1296*x^16 - 139/54*x^15 - 1483/648*x^14 + 7/18*x^13 + 155/54*x^12 + 7/18*x^11 - 1483/648*x^10 - 139/54*x^9 - 35/1296*x^8 + 11/3*x^7 + 509/216*x^6 - 1/6*x^5 - 215/144*x^4 - 17/6*x^3 - 1/2*x^2 + 2*x + 1

In [77]:
#one can factor the polynomial over a splitting field L/K
#but there is no way to express the roots of a quintic in terms of radicals
eigs = matrix(L,U_dft).eigenvalues(extend=False); eigs

TypeError: No compatible natural embeddings found for Number Field in a with defining polynomial x^192 + 192*x^191 + 14112*x^190 + 360144*x^189 - 10547640*x^188 - 759567024*x^187 + 3303157536*x^186 + 1122505191504*x^185 + 5401647864108*x^184 - 1721006003784240*x^183 - 35525503744869888*x^182 + 1677204338167094544*x^181 + 68471564531104940184*x^180 - 810072490769586088560*x^179 - 78750120126358437740064*x^178 - 92202012249342070521456*x^177 + 76961116493830328018951574*x^176 + 1091784078446739644582245008*x^175 - 57016504061931776880623223168*x^174 - 1899974167833213290393577144720*x^173 + 15762506151213242060867303022072*x^172 + 1815943377932309641832033486649840*x^171 + 22487702029300480198395368991239136*x^170 - 924239962930982964793384896135282960*x^169 - 36932060016629518907608609070841454260*x^168 - 112814749564950243986936680722856252560*x^167 + 27434219980661484801871073978391471218304*x^166 + 772271081184566021686911313828834363923888*x^165 - 40333936529460013199893626170878394

In [ ]:
rational_approx_args = [(arg(eig)/(2*pi)).n(100).nearby_rational(max_denominator=100_000_000_000_000_000_000_000) for eig in eigs]; rational_approx_args

In [ ]:
#galois_group() appears to be the absolute version
#NOTE: for a splitting field L, we should have [L:K] = |Gal(L/K)|
if L != L.algebraic_closure():
    L_G = L.galois_group()

In [ ]:
#n=3: two real, two complex
#n=4: all complex
#the magnitude is not 1, they're closely grouped around 2 or 3

In [ ]:
SGA_eigs = SGA_dft.eigenvalues(); SGA_eigs

In [ ]:
[abs(eig) for eig in SGA_eigs]

In [ ]:
#note that the singluar values are the square roots of the diagonal entries of the Gram matrix
print(SymmetricGroup(n).algebra(CDF).dft().SVD()[1].numpy().diagonal())
print(sqrt((SGA_dft*SGA_dft.transpose()).numpy().diagonal()))